In [1]:
import requests

# change to server address
server_url = "http://127.0.0.1:5678/process"

In [2]:
import re 
def parse(text):
    # Return (action_name, xy_touch, xy_lift, type_text)
    action = None
    xy_touch = None
    xy_lift = None
    type_text = None
    bbox_p = re.compile('\[\[([^]]+)\]\]')
    try:
        if len(text) >= 4 and text[:4] == 'type':
            action = 'type'
            type_text = text[4:].strip()
        elif len(text) >= 5 and text[:5] == 'click':
            action = 'click'
            xy_touch = bbox_p.findall(text)[-1]
            xy_touch = xy_touch.replace(' ', '').split(',')
            xy_touch = [int(d) for d in xy_touch]
        elif len(text) >= 10 and text[:10] == 'dual_point':
            action = 'dual_point'
            xy_touch, xy_lift = bbox_p.findall(text)[-2:]
            xy_touch = xy_touch.replace(' ', '').split(',')
            xy_lift = xy_lift.replace(' ', '').split(',')
            xy_touch = [int(d) for d in xy_touch]
            xy_lift = [int(d) for d in xy_lift]
        elif len(text) >= 10 and text[:10] == 'press_back':
            action = 'press_back'
        elif len(text) >= 10 and text[:10] == 'press_home':
            action = 'press_home'
        elif len(text) >= 11 and text[:11] == 'press_enter':
            action = 'press_enter'
        elif len(text) >= 13 and text[:13] == 'task_complete':
            action = 'task_complete'
        elif len(text) >= 15 and text[:15] == 'task_impossible':
            action = 'task_impossible'
        return (action, xy_touch, xy_lift, type_text)
    except:
        return (None, None, None, None)


In [4]:
from ppadb.client import Client as AdbClient

class Controller:
    def __init__(self):
        self.client = AdbClient(host="127.0.0.1", port=5037)
        self.device = self.client.devices()[0]
    def control_device(self, action):
        w, h = 1080, 2400
        if action[0] is None:
            return
        if action[0] == 'type':
            self.device.shell(f"input text '{action[3]}'")
        elif action[0] == 'click':
            self.device.shell(f"input tap {int((action[1][0]+action[1][2])/2*w/1000)} {int((action[1][1]+action[1][3])/2*h/1000)}")
        elif action[0] == 'dual_point':
            if action[1] == action[2]:
                self.device.shell(f"input tap {int((action[1][0]+action[1][2])/2*w/1000)} {int((action[1][1]+action[1][3])/2*h/1000)}")
            self.device.shell(f"input swipe {int((action[1][0]+action[1][2])/2*w/1000)} {int((action[1][1]+action[1][3])/2*h/1000)} {int((action[2][0]+action[2][2])/2*w/1000)} {int((action[2][1]+action[2][3])/2*h/1000)} 250")
        elif action[0] == 'press_back':
            self.device.shell(f"input keyevent KEYCODE_BACK")
        elif action[0] == 'press_home':
            self.device.shell(f"input keyevent KEYCODE_HOME")
        elif action[0] == 'press_enter':
            self.device.shell(f"input keyevent KEYCODE_ENTER")
        else:
            return False
        return True
    def get_image(self, path):
        result = self.device.screencap()
        with open(path, 'wb') as fp:
            fp.write(result)

In [5]:
import matplotlib.pyplot as plt
from PIL import Image
def visualize(image, result):
    img = Image.open(image)
    w, h = img.size
    action_name, xy_touch, xy_lift, type_text = result
    plt.imshow(img)
    plt.title(f'{action_name}')
    if xy_touch is not None:
        touch = ((xy_touch[0]+xy_touch[2])/2000*w, (xy_touch[1]+xy_touch[3])/2000*h)
        plt.plot([touch[0]], [touch[1]], 'ro')
    if xy_lift is not None:
        lift = ((xy_lift[0]+xy_lift[2])/2000*w, (xy_lift[1]+xy_lift[3])/2000*h)
        plt.plot([touch[0], lift[0]], [touch[1], lift[1]], '--b')
    if type_text is not None:
        plt.title(f'Type: {type_text}')
    plt.axis('off')
    plt.show()

In [6]:
import time
cnt = 0
inst = input()
controller = Controller()

while cnt < 20:
    cnt += 1
    print('Instruction:', inst)
    text_data = f"<image>\nPlease provide the bounding box coordinate of the region for this instruction: <ref>{inst}</ref>"

    image_path = "tmp.png"
    controller.get_image(image_path)

    with open(image_path, 'rb') as image_file:
        files = {
            'text': (None, text_data),  
            'image': ('image.jpg', image_file, 'image/jpeg')  
        }

        response = requests.post(server_url, files=files)
     
    result = parse(response.json()['response'])
    #visualize(image_path, result)
    print(result)
    res = controller.control_device(result)
    if not res:
        break
    time.sleep(5)

Instruction: d
('press_home', None, None, None)


KeyboardInterrupt: 